# H4: Графовые признаки

**H4 (Tech):** Графовые признаки по `device_fingerprint`/`ip_address`/`card_number` увеличивают PR-AUC за счёт выявления колец.

In [1]:
import sys
from pathlib import Path
ROOT = Path().resolve()
if not (ROOT/'src').exists(): ROOT = ROOT.parent
sys.path.insert(0, str(ROOT))
print('Project root:', ROOT)


Project root: /Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025


In [2]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.pipeline import Pipeline
from src.data import load_transactions, load_fx
from src.currency import convert_to_usd
from src.features import unpack_last_hour_activity, add_basic_time_features
from src.validation import split_time_aware
from src.pipeline import build_preprocessor, build_logreg
from src.eval import eval_pack
from src.graph_feats import build_graph, graph_features

DATA=Path('../data'); TX=DATA/'transaction_fraud_data.parquet'; FX=DATA/'historical_currency_exchange.parquet'
df = load_transactions(TX); fx = load_fx(FX)
df = convert_to_usd(df, fx); df = unpack_last_hour_activity(df); df = add_basic_time_features(df)

train, test = split_time_aware(df)
y_tr, y_te = train['is_fraud'].astype(int), test['is_fraud'].astype(int)
X_tr, X_te = train.drop(columns=['is_fraud']), test.drop(columns=['is_fraud'])

state = build_graph(train)
g_tr = graph_features(train, state); g_te = graph_features(test, state)
X_tr_g = pd.concat([X_tr.reset_index(drop=True), g_tr.reset_index(drop=True)], axis=1)
X_te_g = pd.concat([X_te.reset_index(drop=True), g_te.reset_index(drop=True)], axis=1)

pipe = Pipeline([('prep', build_preprocessor(X_tr_g)), ('clf', build_logreg())]).fit(X_tr_g, y_tr)
p = pipe.predict_proba(X_te_g)[:,1]
eval_pack(y_te, p)

/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: divide by zero encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: overflow encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)
/Users/gumerovbr/Documents/GitHub/data_analysis_itmo_2025/.venv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:209: RuntimeWarning: invalid value encountered in matmul
  norm2_w = weights @ weights if weights.ndim == 1 else squared_norm(weights)


{'roc_auc': 0.5001122214983779,
 'pr_auc': 0.1980998640895631,
 'threshold_at_precision': 0.9,
 'thr_value': None,
 'recall_at_precision': None,
 'precision_achieved': None}

### Выводы по H4 (графовые признаки)

**Факт**
- Качество на графовых фичах не отличается от случайного: **ROC-AUC ≈ 0.5001**, **PR-AUC ≈ 0.1981**; порог при **Precision ≥ 0.90** не найден. → Текущая реализация графа **не даёт сигнала**.

**Вероятные причины**
1. **Низкая вариативность фичей**: `deg_*` и `comp_size_*` близки к константам (много узлов со степенью 1 и маленькими компонентами).
2. **Сильная «новизна» узлов на тесте**: доля unseen `card/device/ip` высока → фичи маппятся в 0/1 и теряют информативность.
3. **Слишком «узкий» граф**: рёбра только `card–device` и `card–ip`; нет связей с `vendor`, `ip–device`, временных окон.
4. **Агрегация без знания меток**: нет исторических признаков типа «в компоненте уже был fraud до t».

**Мини-диагностика (перед следующей итерацией)**
- Проверить распределения:
  - `X_tr_g[['deg_card','deg_device','deg_ip']].describe()`
  - Долю unseen узлов на тесте: `(~X_te_g['deg_card'].gt(0)).mean()` и аналогично для device/ip.
- Если дисперсии ~0 или unseen >> 50% — текущие фичи по определению будут слабые.

**Рекомендации (следующая итерация)**
1. **Богаче граф и окна во времени**  
   - Добавить рёбра: `device–ip`, `device–vendor`, `ip–vendor`; строить граф **на скользящем окне** (например, 30 дней) для снижения доли unseen.
2. **Исторические риск-агрегаты (без утечки)**  
   - Для узлов/компонент: `prior_fraud_count`, `prior_fraud_rate` до момента t, сглаженные (Laplace).  
   - Флаг «в компоненте уже был fraud до t».
3. **Поведенческие соединения**  
   - `unique_cards_per_device_{1,7,30}d`, `unique_devices_per_ip_{…}`, `first_seen_age_{…}`.
4. **Модель**  
   - Попробовать **LightGBM** (лучше работает с разреженными/не-нормальными распределениями).
5. **Fallback на редкость**  
   - Объединить графовые признаки с усиленными rarity-фичами (`device_is_new/rare`, `ip_is_new/rare`) и их взаимодействиями с CNP/cross-border.

**Итог:** гипотеза в текущей реализации **не подтверждена**. Для появления ценности нужны (а) более «связный» граф с временными окнами и (б) исторические риск-агрегаты без утечки; ожидаемый эффект — рост **PR-AUC** и достижимость целевой **Precision** на сегментах.